# Steps to conduct

1. Setup (Databricks/AWS/GitHub)     ---------------------------                       <-- We are currently here!
2. EDA (within Databricks per PySpark/SQL)
3. Feature Selection (Databricks)
4. Write to S3
5. ML case on AWS Sagemaker (model, scores, etc.)
6. Deployment on AWS Sagemaker
6. Create presentation
7. (20:80 or optional task)

# Import Data

In [0]:
# connect so s3 bucket
# get credentials
import os

ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "aida-project"
MOUNT_NAME = "data"

# mount data
dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3321533958678360> in <module> 
 10 
 11 # mount data 
 ---> 12 dbutils . fs . mount ( "s3a://%s:%s@%s" % ( ACCESS_KEY , ENCODED_SECRET_KEY , AWS_BUCKET_NAME ) , "/mnt/%s" % MOUNT_NAME ) 
 13 display ( dbutils . fs . ls ( "/mnt/%s" % MOUNT_NAME ) ) 

 /local_disk0/tmp/1605600349502-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o269.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:471)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/data
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:220)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:322)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:216)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:77)
	at com.databricks.backend.daemon.data.server.handler.CEMountHandler.receive(MountHandler.scala:122)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:354)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:322)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:432)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	

In [0]:
display(dbutils.fs.ls("/mnt/%s/TSV" % MOUNT_NAME))

In [0]:
df_names = spark.read.load("dbfs:/mnt/data/TSV/name.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_akas = spark.read.load("dbfs:/mnt/data/TSV/title.akas.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_basics = spark.read.load("dbfs:/mnt/data/TSV/title.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_principals = spark.read.load("dbfs:/mnt/data/TSV/title.principals.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_ratings = spark.read.load("dbfs:/mnt/data/TSV/title.ratings.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")

In [0]:
list_dfs = [df_names, df_akas, df_basics, df_principals, df_ratings]

for df in list_dfs:
  df.printSchema()

# EDA

## Possible list of questions (non-exhaustive):
1. What is the range of our ratings (inlcuding distribution)? --------------------------------------- DONE
2. How many votes does a movie have on average? (Box-Plot)
3. What timeframe does our dataset span? (e.g. oldest and newest movie)
4. Who are the most popular actors and directors?
5. What genres are represented the most?
6. Distribution of films / series / shows?
    ----> Focus on movies!
7. What genres are in the dataset?
8. Which genres have the highest rating?
9. Which actors play in the high rated films?
10. In which countries were the most high-rated films made and when?
11. Dependence on high rated film:
    - Country of origin ---> Actor?
    - Genre -> Actor
    - Year of creation -> Actor -> Genre
12. Which parameters go into the rating?

.....

## Imports and Functions

In [0]:
from pyspark.sql.functions import mean as _mean, \
                                  min as _min, \
                                  max as _max, \
                                  count as _count, \
                                  stddev as _stddev, col

import matplotlib.pyplot as plt
from pyspark.sql import functions as F

In [0]:
percentiles = [0.1, 0.25, 0.5, 0.75, 0.9]

In [0]:
# https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.histogram

def viz_histogram(dataframe, column, buckets):
  bins, counts = dataframe.select(column).rdd.flatMap(lambda x: x).histogram(buckets)
  plt.hist(bins[:-1], bins=bins, weights=counts)
  plt.title(f'Histogram of {column}')
  
  
def show_quantiles(dataframe, list_columns, percentiles):
  df_quantile = spark.createDataFrame(
      zip(percentiles, *df.approxQuantile(columns, percentiles, 0.1)), 
      ["Pecentile"] + columns
  )
  df_quantile.show()
  
  return df_quantile

## What is the range of our ratings (inlcuding distribution)

In [0]:
stats_ratings = df_ratings.select(
    _mean(col('averageRating')).alias('mean'),
    _min(col('averageRating')).alias('min'),
    _max(col('averageRating')).alias('max'),
    _stddev(col('averageRating')).alias('std')).collect()

min_ratings = stats_ratings[0]['min']
mean_ratings = stats_ratings[0]['mean']
max_ratings = stats_ratings[0]['max']
std_ratings = stats_ratings[0]['std']

stats_ratings

In [0]:
df_ratings.describe().show()

In [0]:
viz_histogram(df_ratings, 'averageRating', 9)

Our ratings range from **1** to **10**. The mean rating is **6.89** (maybe people are a little bit kinder than expected), with a standard deviation of roughly **1.40**. We have roughly **1 million** entries.

## How many votes does a movie have on average? (Box-Plot)

In [0]:
df_ratings.groupBy('tconst').mean('numVotes').describe().show()

In [0]:
quantiles_votes = show_quantiles(df_ratings, 'numVotes', percentiles)

In [0]:
viz_histogram(df_ratings, 'numVotes', 1000)

# Feature Selection & Data Cleaning

# Write to S3

In [0]:
# TO DEFINE SHAPE OF TARGET DATASET

# id, rating, [list of features]

# sample of target dataset (e.g. 20%) write to s3

df_final.write('s3')

In [0]:
spark.version

In [0]:
#Import of the dataset

df_basics_original = spark.read.load("dbfs:/mnt/data/TSV/title.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")




In [0]:
df_basis = df_basics_original.sample(False, 0.5, 42)

In [0]:
# What kind of data are in the columns
df_basics.printSchema()

root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- primaryTitle: string (nullable = true)
-- originalTitle: string (nullable = true)
-- isAdult: integer (nullable = true)
-- startYear: string (nullable = true)
-- endYear: string (nullable = true)
-- runtimeMinutes: string (nullable = true)
-- genres: string (nullable = true)

In [0]:
display(df_basics)

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
tt0000010,short,Exiting the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [0]:
display(df_basics.describe())

summary,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,6326545,6326545,6326545,6326545,6326545,6326545,6326545,6326545,6326535
mean,null,null,NaN,NaN,0.033084724758932406,2001.4857464536149,2001.6832648116879,45.00799321813517,null
stddev,null,null,NaN,NaN,2.536020251863344,21.183997012090863,18.883630674696988,76.33630531590723,null
min,tt0000001,movie,!Next?,!Next?,0,1874,1924,0,Action
max,tt9916880,videoGame,Šiška Deluxe,üç,2019,\N,\N,\N,\N


In [0]:
df_basics.show(vertical=True)

-RECORD 0------------------------------
 tconst | tt0000001 
 titleType | short 
 primaryTitle | Carmencita 
 originalTitle | Carmencita 
 isAdult | 0 
 startYear | 1894 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 1------------------------------
 tconst | tt0000002 
 titleType | short 
 primaryTitle | Le clown et ses c... 
 originalTitle | Le clown et ses c... 
 isAdult | 0 
 startYear | 1892 
 endYear | \N 
 runtimeMinutes | 5 
 genres | Animation,Short 
-RECORD 2------------------------------
 tconst | tt0000003 
 titleType | short 
 primaryTitle | Pauvre Pierrot 
 originalTitle | Pauvre Pierrot 
 isAdult | 0 
 startYear | 1892 
 endYear | \N 
 runtimeMinutes | 4 
 genres | Animation,Comedy,... 
-RECORD 3------------------------------
 tconst | tt0000004 
 titleType | short 
 primaryTitle | Un bon bock 
 originalTitle | Un bon bock 
 isAdult | 0 
 startYear | 1892 
 endYear | \N 
 runtimeMinutes | \N 
 genres | Animation,Short 
-RECORD 4------------------------------
 tconst | tt0000005 
 titleType | short 
 primaryTitle | Blacksmith Scene 
 originalTitle | Blacksmith Scene 
 isAdult | 0 
 startYear | 1893 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Comedy,Short 
-RECORD 5------------------------------
 tconst | tt0000006 
 titleType | short 
 primaryTitle | Chinese Opium Den 
 originalTitle | Chinese Opium Den 
 isAdult | 0 
 startYear | 1894 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Short 
-RECORD 6------------------------------
 tconst | tt0000007 
 titleType | short 
 primaryTitle | Corbett and Court... 
 originalTitle | Corbett and Court... 
 isAdult | 0 
 startYear | 1894 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Short,Sport 
-RECORD 7------------------------------
 tconst | tt0000008 
 titleType | short 
 primaryTitle | Edison Kinetoscop... 
 originalTitle | Edison Kinetoscop... 
 isAdult | 0 
 startYear | 1894 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 8------------------------------
 tconst | tt0000009 
 titleType | movie 
 primaryTitle | Miss Jerry 
 originalTitle | Miss Jerry 
 isAdult | 0 
 startYear | 1894 
 endYear | \N 
 runtimeMinutes | 45 
 genres | Romance 
-RECORD 9------------------------------
 tconst | tt0000010 
 titleType | short 
 primaryTitle | Exiting the Factory 
 originalTitle | La sortie de l'us... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 10-----------------------------
 tconst | tt0000011 
 titleType | short 
 primaryTitle | Akrobatisches Pot... 
 originalTitle | Akrobatisches Pot... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 11-----------------------------
 tconst | tt0000012 
 titleType | short 
 primaryTitle | The Arrival of a ... 
 originalTitle | L'arrivée d'un tr... 
 isAdult | 0 
 startYear | 1896 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Action,Documentar... 
-RECORD 12-----------------------------
 tconst | tt0000013 
 titleType | short 
 primaryTitle | The Photographica... 
 originalTitle | Neuville-sur-Saôn... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 13-----------------------------
 tconst | tt0000014 
 titleType | short 
 primaryTitle | The Sprinkler Spr... 
 originalTitle | L'arroseur arrosé 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Comedy,Short 
-RECORD 14-----------------------------
 tconst | tt0000015 
 titleType | short 
 primaryTitle | Autour d'une cabine 
 originalTitle | Autour d'une cabine 
 isAdult | 0 
 startYear | 1894 
 endYear | \N 
 runtimeMinutes | 2 
 genres | Animation,Short 
-RECORD 15-----------------------------
 tconst | tt0000016 
 titleType | short 
 primaryTitle | Barque sortant du... 
 originalTitle | Barque sortant du... 
 isAdult | 0 
 startYear | 1895 
 endYear | \N 
 runtimeMinutes | 1 
 genres | Documentary,Short 
-RECORD 16--------------------

- Start all tconst with tt?
- Only 'isAdult' is an integer. What is the range or which values are available? (1)
- What is the content of titleTyp? (2)
- What is the difference between titleTyp and genres?
- What means nullable=true?
- Column 'genres' contains different types of genres - seperate!

(1) Only 'isAdult' is an integer. What is the range or which values are available?

In [0]:
display(
  df_basics.select('isAdult').distinct()
)

isAdult
1
0
2019
2004
2014
2018
2015
1994
2005


In [0]:
# Distribution of 0, 1, and the year numbers
groupBy_output = df_basics.groupBy("isAdult").count()
display(groupBy_output)

isAdult,count
1,189186
0,6137349
2019,4
2004,1
2014,1
2018,1
2015,1
1994,1
2005,1


Content of the column isAdult contain the values 0 and 1,sometimes a year number ----> delete all year number entries!

In [0]:
# GroupBy the titleTyp and count the sum

groupBy_output = df_basics.groupBy("titleType").count()
display(groupBy_output)

titleType,count
tvSeries,174601
tvMiniSeries,28413
tvMovie,120424
tvEpisode,4445415
movie,536248
tvSpecial,26137
video,247411
videoGame,24550
tvShort,11569
short,711777


In [0]:
groupBy_output = df_basics.groupBy("titleType").count()
display(groupBy_output)

titleType,count
tvSeries,174601
tvMiniSeries,28413
tvMovie,120424
tvEpisode,4445415
movie,536248
tvSpecial,26137
video,247411
videoGame,24550
tvShort,11569
short,711777


---> most of the movies are in the titleType tvEpisode

In [0]:
# GroupBy the genres and count the sum

groupBy_output = df_basics.groupBy("titleType", "genres","isAdult").count()
display(groupBy_output)

titleType,genres,isAdult,count
short,"Biography,Romance,Short",0,17
movie,"Fantasy,Sci-Fi",0,117
movie,"Comedy,Family,Musical",0,89
movie,"Film-Noir,Mystery,Thriller",0,6
movie,"Animation,Documentary,History",0,48
tvSeries,Comedy,0,26951
tvSeries,"Adventure,Mystery",0,13
movie,"Action,Horror,Thriller",0,271
tvSeries,"Animation,Family,Western",0,3
tvSeries,"Adventure,Drama,Sci-Fi",0,24


In [0]:
from pyspark.sql.functions import split


In [0]:
# Split of the genres column in genre 1, 2, 3

split_col = split(df_basics['genres'], ',')
df_basics_test = df_basics.withColumn('NAME1', split_col.getItem(0))
df_basics_test = df_basics.withColumn('NAME2', split_col.getItem(1))
df_basics_test = df_basics.withColumn('NAME3', split_col.getItem(2))

df_basics.display()

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short"
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
tt0000010,short,Exiting the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"


In [0]:
# Alternative: Split of the genres column in genre 1, 2, 3
df_basics_encode = df_basics.withColumn("genre1", split(col("genres"), ",").getItem(0)).withColumn("genre2", split(col("genres"), ",").getItem(1)).withColumn("genre3", split(col("genres"), ",").getItem(2))
df_basics_encode.display()

tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre1,genre2,genre3
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",Documentary,Short,null
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",Animation,Short,null
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",Animation,Comedy,Romance
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",Animation,Short,null
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",Comedy,Short,null
tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short,Short,null,null
tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport",Short,Sport,null
tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",Documentary,Short,null
tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,Romance,null,null
tt0000010,short,Exiting the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short",Documentary,Short,null


In [0]:
input = df_basics_encode['genre1', 'genre1', 'genre',], 
#output =['genre1index']
input


--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3955913033488581> in <module> 
 1 input = df_basics_encode [ 'genre1' , 'genre2' , 'genre3' ] , 
 ----> 2 output = df_basics_encode [ 'genre1index' , 'genre2index' , 'genre3index' ] 
 3 output

 /databricks/spark/python/pyspark/sql/dataframe.py in __getitem__ (self, item) 
 1401 return self . filter ( item ) 
 1402 elif isinstance ( item , ( list , tuple ) ) : 
 -> 1403 return self . select ( * item ) 
 1404 elif isinstance ( item , int ) : 
 1405 jc = self . _jdf . apply ( self . columns [ item ] ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1437 [ Row ( name = u'Alice' , age = 12 ) , Row ( name = u'Bob' , age = 15 ) ] 
 1438 """
 -> 1439 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1440 return DataFrame ( jdf , self . sql_ctx ) 
 1441 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : cannot resolve '`genre1index`' given input columns: [endYear, genre1, genre2, genre3, genres, isAdult, originalTitle, primaryTitle, runtimeMinutes, startYear, tconst, titleType];;
'Project ['genre1index, 'genre2index, 'genre3index]
+- Project [tconst#10949, titleType#10950, primaryTitle#10951, originalTitle#10952, isAdult#10953, startYear#10954, endYear#10955, runtimeMinutes#10956, genres#10957, genre1#18452, genre2#18463, split(genres#10957, ,, -1)[2] AS genre3#18475]
 +- Project [tconst#10949, titleType#10950, primaryTitle#10951, originalTitle#10952, isAdult#10953, startYear#10954, endYear#10955, runtimeMinutes#10956, genres#10957, genre1#18452, split(genres#10957, ,, -1)[1] AS genre2#18463]
 +- Project [tconst#10949, titleType#10950, primaryTitle#10951, originalTitle#10952, isAdult#10953, startYear#10954, endYear#10955, runtimeMinutes#10956, genres#10957, split(genres#10957, ,, -1)[0] AS genre1#18452]
 +- Relation[tconst#10949,titleType#10950,primaryTitle#10951,originalTitle#10952,isAdult#10953,startYear#10954,endYear#10955,runtimeMinutes#10956,genres#10957] csv

In [0]:
# Transform genre columns into StringIndexer

from pyspark.ml.feature import StringIndexer


#input = df_basics_encode['genre1']

indexer = StringIndexer(inputCol='genre1', outputCol='genre1index')



indexed = indexer.fit(df_basics_encode).transform(df_basics_encode)
indexed.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-----------+-------+-----------+
 tconst|titleType| primaryTitle| originalTitle|isAdult|startYear|endYear|runtimeMinutes| genres| genre1| genre2| genre3|genre1index|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-----------+-------+-----------+
tt0000001| short| Carmencita| Carmencita| 0| 1894| \N| 1| Documentary,Short|Documentary| Short| null| 2.0|
tt0000002| short|Le clown et ses c...|Le clown et ses c...| 0| 1892| \N| 5| Animation,Short| Animation| Short| null| 7.0|
tt0000003| short| Pauvre Pierrot| Pauvre Pierrot| 0| 1892| \N| 4|Animation,Comedy,...| Animation| Comedy|Romance| 7.0|
tt0000004| short| Un bon bock| Un bon bock| 0| 1892| \N| \N| Animation,Short| Animation| Short| null| 7.0|
tt0000005| short| Blacksmith Scene| Blacksmith Scene| 0| 1893| \N| 1| Comedy,Short| Comedy| Short| null| 1.0|
tt0000006| short| Chinese Opium Den| Chinese Opium Den| 0| 1894| \N| 1| Short| Short| null| null| 12.0|
tt0000007| short|Corbett and Court...|Corbett and Court...| 0| 1894| \N| 1| Short,Sport| Short| Sport| null| 12.0|
tt0000008| short|Edison Kinetoscop...|Edison Kinetoscop...| 0| 1894| \N| 1| Documentary,Short|Documentary| Short| null| 2.0|
tt0000009| movie| Miss Jerry| Miss Jerry| 0| 1894| \N| 45| Romance| Romance| null| null| 17.0|
tt0000010| short| Exiting the Factory|La sortie de l'us...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0|
tt0000011| short|Akrobatisches Pot...|Akrobatisches Pot...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0|
tt0000012| short|The Arrival of a ...|L'arrivée d'un tr...| 0| 1896| \N| 1|Action,Documentar...| Action|Documentary| Short| 6.0|
tt0000013| short|The Photographica...|Neuville-sur-Saôn...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0|
tt0000014| short|The Sprinkler Spr...| L'arroseur arrosé| 0| 1895| \N| 1| Comedy,Short| Comedy| Short| null| 1.0|
tt0000015| short| Autour d'une cabine| Autour d'une cabine| 0| 1894| \N| 2| Animation,Short| Animation| Short| null| 7.0|
tt0000016| short|Barque sortant du...|Barque sortant du...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0|
tt0000017| short|Italienischer Bau...|Italienischer Bau...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0|
tt0000018| short|Das boxende Känguruh|Das boxende Känguruh| 0| 1895| \N| 1| Short| Short| null| null| 12.0|
tt0000019| short| The Clown Barber| The Clown Barber| 0| 1898| \N| \N| Comedy,Short| Comedy| Short| null| 1.0|
tt0000020| short| The Derby 1895| The Derby 1895| 0| 1895| \N| 1|Documentary,Short...|Documentary| Short| Sport| 2.0|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-----------+-------+-----------+
only showing top 20 rows

In [0]:
# OneHotEncoding of genre1, genre2 and genre3

from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["genre1index"],
                        outputCols=["genral1index-onehot"])
model = encoder.fit(indexed)
encoded = model.transform(indexed)
encoded.show()




+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-----------+-------+-----------+-------------------+
 tconst|titleType| primaryTitle| originalTitle|isAdult|startYear|endYear|runtimeMinutes| genres| genre1| genre2| genre3|genre1index|genral1index-onehot|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-----------+-------+-----------+-------------------+
tt0000001| short| Carmencita| Carmencita| 0| 1894| \N| 1| Documentary,Short|Documentary| Short| null| 2.0| (28,[2],[1.0])|
tt0000002| short|Le clown et ses c...|Le clown et ses c...| 0| 1892| \N| 5| Animation,Short| Animation| Short| null| 7.0| (28,[7],[1.0])|
tt0000003| short| Pauvre Pierrot| Pauvre Pierrot| 0| 1892| \N| 4|Animation,Comedy,...| Animation| Comedy|Romance| 7.0| (28,[7],[1.0])|
tt0000004| short| Un bon bock| Un bon bock| 0| 1892| \N| \N| Animation,Short| Animation| Short| null| 7.0| (28,[7],[1.0])|
tt0000005| short| Blacksmith Scene| Blacksmith Scene| 0| 1893| \N| 1| Comedy,Short| Comedy| Short| null| 1.0| (28,[1],[1.0])|
tt0000006| short| Chinese Opium Den| Chinese Opium Den| 0| 1894| \N| 1| Short| Short| null| null| 12.0| (28,[12],[1.0])|
tt0000007| short|Corbett and Court...|Corbett and Court...| 0| 1894| \N| 1| Short,Sport| Short| Sport| null| 12.0| (28,[12],[1.0])|
tt0000008| short|Edison Kinetoscop...|Edison Kinetoscop...| 0| 1894| \N| 1| Documentary,Short|Documentary| Short| null| 2.0| (28,[2],[1.0])|
tt0000009| movie| Miss Jerry| Miss Jerry| 0| 1894| \N| 45| Romance| Romance| null| null| 17.0| (28,[17],[1.0])|
tt0000010| short| Exiting the Factory|La sortie de l'us...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0| (28,[2],[1.0])|
tt0000011| short|Akrobatisches Pot...|Akrobatisches Pot...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0| (28,[2],[1.0])|
tt0000012| short|The Arrival of a ...|L'arrivée d'un tr...| 0| 1896| \N| 1|Action,Documentar...| Action|Documentary| Short| 6.0| (28,[6],[1.0])|
tt0000013| short|The Photographica...|Neuville-sur-Saôn...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0| (28,[2],[1.0])|
tt0000014| short|The Sprinkler Spr...| L'arroseur arrosé| 0| 1895| \N| 1| Comedy,Short| Comedy| Short| null| 1.0| (28,[1],[1.0])|
tt0000015| short| Autour d'une cabine| Autour d'une cabine| 0| 1894| \N| 2| Animation,Short| Animation| Short| null| 7.0| (28,[7],[1.0])|
tt0000016| short|Barque sortant du...|Barque sortant du...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0| (28,[2],[1.0])|
tt0000017| short|Italienischer Bau...|Italienischer Bau...| 0| 1895| \N| 1| Documentary,Short|Documentary| Short| null| 2.0| (28,[2],[1.0])|
tt0000018| short|Das boxende Känguruh|Das boxende Känguruh| 0| 1895| \N| 1| Short| Short| null| null| 12.0| (28,[12],[1.0])|
tt0000019| short| The Clown Barber| The Clown Barber| 0| 1898| \N| \N| Comedy,Short| Comedy| Short| null| 1.0| (28,[1],[1.0])|
tt0000020| short| The Derby 1895| The Derby 1895| 0| 1895| \N| 1|Documentary,Short...|Documentary| Short| Sport| 2.0| (28,[2],[1.0])|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-----------+-----------+-------+-----------+-------------------+
only showing top 20 rows

In [0]:
df_basics_encode.printSchema()

root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- primaryTitle: string (nullable = true)
-- originalTitle: string (nullable = true)
-- isAdult: integer (nullable = true)
-- startYear: string (nullable = true)
-- endYear: string (nullable = true)
-- runtimeMinutes: string (nullable = true)
-- genres: string (nullable = true)
-- genre1: string (nullable = true)
-- genre2: string (nullable = true)
-- genre3: string (nullable = true)

Next steps:

root
- |-- tconst: string (nullable = true) 
- |-- titleType: string (nullable = true)  ---> delete NaN
- |-- primaryTitle: string (nullable = true) ---> delete NaN
- |-- originalTitle: string (nullable = true)---> delete NaN
- |-- isAdult: integer (nullable = true)---> delete year numbers
- |-- startYear: string (nullable = true)---> delete /n
- |-- endYear: string (nullable = true)---> delete /n
- |-- runtimeMinutes: string (nullable = true)---> delete /n
- |-- genres: string (nullable = true)---> delete /n
- |-- genre1: string (nullable = true)---> delete /n
- |-- genre2: string (nullable = true)---> delete /n
- |-- genre3: string (nullable = true)---> delete /n


- Encoding of Genres